In [1]:
## NowPlaying Cultural Sentiment Data
"""
cs_header = ["hashtag", "AFINN_min", "AFINN_max", "AFINN_sum", "AFINN_avg", "OpinionLex_min", "OpinionLex_max", "OpinionLex_sum", "OpinionLex_avg", "Sentistrength_min", "Sentistrength_max", "Sentistrength_sum", "Sentistrength_avg", "Vader_min", "Vader_max", "Vader_sum", "Vader_avg", "SentimentHashtag_min", "SentimentHashtag_max", "SentimentHashtag_sum", "SentimentHashtag_avg"]
np_cs = pd.read_csv('np_cultural_sentiment.csv',names = cs_header)
"""
#IMPORTING Cultural Sentiment Data to MongoDB
"""
np_cs_json = json.loads(np_cs.to_json(orient="records"))
db.cultural_sentiment.drop()
db.cultural_sentiment.insert_many(np_cs_json)
"""
## Transformation of NowPlaying Cultural Tweets to DataframeStructure
"""
output = []
with open("np_cultural.json") as f:
    for line in f:     
        output.append(json.load|s(line))
np_c = pd.DataFrame(output)
"""
#IMPORTING Cultural Tweets to MongoDB
"""
db.cultural_tweet.drop()
records = json.loads(np_c.T.to_json()).values()
db.cultural_tweet.insert_many(records)
"""

## Transformation and Importing of Playlist Data from NowPlaying to MongoDB
"""
np_pl = pd.read_csv("playlist.csv", names=["playlist_id","track_id","playlist_name"])
np_pl_json = json.loads(np_pl.to_json(orient="records"))
db.playlist.drop()
db.playlist.insert_many(np_pl_json)
"""

'\nnp_pl = pd.read_csv("playlist.csv", names=["playlist_id","track_id","playlist_name"])\nnp_pl_json = json.loads(np_pl.to_json(orient="records"))\ndb.playlist.drop()\ndb.playlist.insert_many(np_pl_json)\n'

# CULTURAL #NOWPLAYING DATA
*********************************

This dataset contains data describing the listening events of users (extracted from the #nowplaying dataset), the emotion extracted from the hashtags used in the according tweets and information about the location of the user.


TWITTER AND TRACK DATA
-------------------------------
The data regarding the listening events is contained in np_cultural.zip and is encoded as json. It holds the following information:

- id: the id of the underlying tweet [*]
- user_id: the id of the user who sent the tweet (MD5 of it)
- user_lang: The BCP 47 code for the user’s self-declared user interface language. [*]
- user_time_zone: [*]
- text: actual content of the tweet [*]
- tweet_lang: language of the tweet (as detected by Twitter; BCP 47 language identifier corresponding to the machine-detected language of the Tweet text, or und if no language could be detected.) [*]
- geo: Deprecated version of coordinates (however, we deal with data stemming from before this API change, therefore we still include it; cf. coordinates for a description)
- coordinates: Represents the geographic location of this Tweet as reported by the user or client application. The inner coordinates array is formatted as geoJSON (longitude first, then latitude). [*]
- place: When present, indicates that the tweet is associated (but not necessarily originating from) a Place. [*]
- created_at: time the tweet was sent. [*]
- source: Utility used to post the Tweet, as an HTML-formatted string. [*]
- track_title: title of the track the user tweeted about
- track_id: the unique id of the track (from #nowplaying dataset) 
- artist_name: name of artist performing the track
- artist_id: the unique id of the artist (from #nowplaying dataset) 
- hashtags: list of hashtags used in the tweet.

[*] for further information about the information gathered from Twitter, please consult https://dev.twitter.com/overview/api/tweets

Please note that we do only add key-value pairs for geo/coordinates/place information if this information was provided by the Twitter API (i.e., missing keys signal that this information is not available for the given tweet).


SENTIMENT DATA
-------------------------------
The data regarding the hashtag's sentiment (if any could be obtained) is contained in np_cultural_sentiment.csv and is formatted as csv. The sentiment score was obtained by applying a set of well-known sentiment dictionaries. The sentiment scores are scaled between 0 and 1 (very negative to very positive). For each dictionary, we list the minimum, maximum, sum and average sentiment score across all hashtags used within every tweet (most tweets only feature a single hashtag we can assign a sentiment value to)
It contains the following information (in this particular order):
- name of the hashtag
- AFINN dictionary (min, max, sum, avg)
- Opinion Lexicon (min, max, sum, avg)
- Sentistrength Lexicon (min, max, sum, avg)
- Vader (min, max, sum, avg)
- Sentiment Hashtag Lexicon (min, max, sum, avg)


Please note that we only added hashtags for which we could obtain a sentiment value from at least one sentiment dictionary.

In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np 
import json
import csv
import pprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from difflib import SequenceMatcher


In [15]:
#Call Mongo Instance
client = MongoClient()
db = client.now_playing

#Spotify Instance
client_data = {}
with open("secrets.json", 'r') as f:
    client_data = json.load(f)
client_data

client_credentials_manager = SpotifyClientCredentials(client_id=client_data["client_id"], client_secret=client_data["client_secret"],)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Importing the MusicBrainz Mapping of artists and tracks and tracks from cultural dataset

`mongoimport -d now_playing -c mb_tracks --type csv --file mb_tracks.csv --fields "np_id,mb_track_id"`

`mongoimport -d now_playing -c mb_artists --type csv --file mb_artists.csv --fields "np_id,mb_artist_id"`

`mongoimport -d now_playing -c np_cultural_mb_tracks --type csv --file np_cultural_mb_tracks.csv --headerline
`

In [21]:
user_ids = db.cultural_tweet.distinct("user_id")
type(user_ids)


list

In [22]:
#db.cultural_tweet.find({"hashtag": "nobeats"})
user_tweets = db.cultural_tweet.find({"user_id": '1c10f9788fdcc4baf6cf6a2631fe78bc12102418'})
#TOTAL Number of Tweets


In [23]:
first = user_ids[0]
user_songs = db.cultural_tweet.find({"user_id": first})
initial_song = user_songs[0]
initial_date = initial_song["created_at"]

In [24]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
def convertMillis(millis):
     seconds = (millis/1000)%60
     minutes = (millis/(1000*60))%60
     return str(int(minutes)) + ":" +str(int(seconds))
    
def get_track_list(find_query):
    return db.cultural_tweet.find(find_query).distinct("track_id")

def get_complete_track(track):
    song_name = track["name"].lower()
    artist_name = " ".join(list(map(lambda x: x["name"], track["artists"]))).lower()
    return song_name + " " + artist_name

def get_searched_track_name(track):
    song_name = db.cultural_tweet.find_one({"track_id" : track}, {"track_title" : 1, "artist_name" : 1})
    return song_name["track_title"].lower() + " " + song_name["artist_name"].lower()


In [ ]:
song_duration = -1
name_found = ""
searched_name = dict()
spotify_id = -1
track_list = get_track_list({"name_found" : {"$exists" : False }, "spotify_not_found" : {"$exists" : False }})
tracks_size = len(track_list)
searched = 0

for track in track_list:
    search_query = get_searched_track_name(track)
    print(search_query)

the the the the
abriendo puertas gloria estefan
the love a tribe called quest
day stevie wonder
come to the teacher boogie down productions
tap the bottle young black teenagers
afterlife stef
salah jatuh cinta gita gutawa
what if creed
going to heaven to meet the king carlton pearson
sweating bullets megadeth
prelude above & beyond
g larsen
casas de cartón marco antonio solís
g so
beer for breakfast jb and the moonshine band
involve yourself avant
got to give it up aaliyah
halt mich lacrimosa
wish luna sea
rej âme
tell her about it billy joel
welcome to los santos oh no
hot avril lavigne
deep in vogue malcolm mclaren
super girl super junior m
nutshell alice in chains
try to find me gorky park
the cold song klaus nomi
bone it like you own it denise lasalle
true blue luna sea
real and true future
resurection ппк
more and more hits nice & smooth
undeniable mat kearney
cant stop wont stop usher
changes yes
before today everything but the girl
sanity assassin bauhaus
stoned and starving par

In [ ]:

for track in track_list:
    search_query = get_searched_track_name(track)
    search_result = sp.search(q = search_query, limit= 3, type="track")
    print(search_result)
    searched += 1
    score = 0
    for t in search_result["tracks"]["items"]:
        result_matcher = get_complete_track(t)
        similarity_score = similar(search_query,result_matcher)
        if(similarity_score > score):
            print("Found " + result_matcher + " => Similarity Score: " + str(similarity_score))
            song_duration = t["duration_ms"]
            score = similarity_score
            name_found = result_matcher
            searched_name[track] = search_query
            spotify_id = t["id"]
    print_text = ""
    if score > 0:
        db.cultural_tweet.update_many({"track_id": track},
        {"$set" : {
            "duration_ms" : song_duration,
            "spotify_id" : spotify_id,
            "similarity_score" : score,
            "name_found" : name_found
        }})
        print_text = " Saved song "+ name_found
    else:
         db.cultural_tweet.update_many({"track_id": track},
         {"$set" : {
             "spotify_not_found" : True
         }})
         print_text = " Not found in Spotify"
    print("Completed " + str(searched) +" out of " + str(tracks_size) + print_text)         
print("saved!")

In [ ]:
clean_dic = {track: final_result[track] for track in final_result if score_result[track] <= 0.80}

In [ ]:
print("Songs with duration " + str(len(db.cultural_tweet.find({"duration_ms" : {"$exists" : True }}).distinct("track_id"))))
print("Songs without duration " + str(len(db.cultural_tweet.find({"duration_ms" : {"$exists" : False}}).distinct("track_id"))))
print("Songs in total " + str(len(db.cultural_tweet.distinct("track_id"))))


In [54]:
test[0]

{u'album': {u'album_type': u'single',
  u'artists': [{u'external_urls': {u'spotify': u'https://open.spotify.com/artist/4V8Sr092TqfHkfAA5fXXqG'},
    u'href': u'https://api.spotify.com/v1/artists/4V8Sr092TqfHkfAA5fXXqG',
    u'id': u'4V8Sr092TqfHkfAA5fXXqG',
    u'name': u'Luis Fonsi',
    u'type': u'artist',
    u'uri': u'spotify:artist:4V8Sr092TqfHkfAA5fXXqG'},
   {u'external_urls': {u'spotify': u'https://open.spotify.com/artist/4VMYDCV2IEDYJArk749S6m'},
    u'href': u'https://api.spotify.com/v1/artists/4VMYDCV2IEDYJArk749S6m',
    u'id': u'4VMYDCV2IEDYJArk749S6m',
    u'name': u'Daddy Yankee',
    u'type': u'artist',
    u'uri': u'spotify:artist:4VMYDCV2IEDYJArk749S6m'}],
  u'available_markets': [u'CA', u'MX', u'US'],
  u'external_urls': {u'spotify': u'https://open.spotify.com/album/3Gq2Dme9nesdgoqNNlcN8O'},
  u'href': u'https://api.spotify.com/v1/albums/3Gq2Dme9nesdgoqNNlcN8O',
  u'id': u'3Gq2Dme9nesdgoqNNlcN8O',
  u'images': [{u'height': 640,
    u'url': u'https://i.scdn.co/image/e